# About

test notebook to explore spacy and bag of words


In [35]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from bs4 import BeautifulSoup
import collections
from typing import Dict, List, Tuple

# Loads Data

In [ ]:
# Loading data, see yuqing's notebook

p_comments = "/mnt/disks/disk-1-w210-data/data/interim/stackexchange/stackoverflow/CommentsFiltered_v3.tsv"
p_tags = "/mnt/disks/disk-1-w210-data/data/interim/stackexchange/stackoverflow/tags.csv"
p_answers = "/mnt/disks/disk-1-w210-data/data/interim/stackexchange/stackoverflow/PostAnswersFiltered_V3_parsed.tsv"
p_questions = "/mnt/disks/disk-1-w210-data/data/interim/stackexchange/stackoverflow/PostQuestionsFiltered_V3_parsed.tsv"

comments = pd.read_csv(p_comments, delimiter = "\t", encoding = "utf-8")
tags = pd.read_csv(p_tags)
answers = pd.read_csv(p_answers, delimiter = "\t", encoding = "utf-8")
questions = pd.read_csv(p_questions, delimiter = "\t", encoding = "utf-8")

In [3]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1184056 entries, 0 to 1184055
Data columns (total 8 columns):
Unnamed: 0           1184056 non-null int64
id                   1184056 non-null int64
text                 1184056 non-null object
creation_date        1184056 non-null object
post_id              1184056 non-null int64
user_id              1171470 non-null float64
user_display_name    12669 non-null object
score                1184056 non-null int64
dtypes: float64(1), int64(4), object(3)
memory usage: 72.3+ MB


In [4]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55665 entries, 0 to 55664
Data columns (total 5 columns):
_Id               55665 non-null int64
_TagName          55663 non-null object
_Count            55665 non-null int64
_ExcerptPostId    41059 non-null float64
_WikiPostId       41059 non-null float64
dtypes: float64(2), int64(2), object(1)
memory usage: 2.1+ MB


In [5]:
answers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889576 entries, 0 to 889575
Data columns (total 26 columns):
Unnamed: 0                  889576 non-null int64
id                          889576 non-null int64
title                       0 non-null float64
body                        889576 non-null object
accepted_answer_id          0 non-null float64
answer_count                0 non-null float64
comment_count               889576 non-null int64
community_owned_date        2623 non-null object
creation_date               889576 non-null object
favorite_count              0 non-null float64
last_activity_date          889576 non-null object
last_edit_date              272709 non-null object
last_editor_display_name    3243 non-null object
last_editor_user_id         271211 non-null float64
owner_display_name          15159 non-null object
owner_user_id               883321 non-null float64
parent_id                   889576 non-null int64
post_type_id                889576 non-null i

In [6]:
questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692177 entries, 0 to 692176
Data columns (total 72 columns):
Unnamed: 0                  692177 non-null int64
Unnamed: 0.1                692177 non-null int64
id                          692177 non-null int64
title                       692177 non-null object
body                        692177 non-null object
accepted_answer_id          353530 non-null float64
answer_count                692177 non-null int64
comment_count               692177 non-null int64
community_owned_date        293 non-null object
creation_date               692177 non-null object
favorite_count              170080 non-null float64
last_activity_date          692177 non-null object
last_edit_date              375266 non-null object
last_editor_display_name    5267 non-null object
last_editor_user_id         371649 non-null float64
owner_display_name          11995 non-null object
owner_user_id               683235 non-null float64
parent_id                   0

## sample

In [7]:
n = 6000
comments_sample = comments.sample(n)
tags_sample = tags.sample(n)
answers_sample = answers.sample(n)
questions_sample = questions.sample(n)

In [8]:
comments_sample.head(3)

,Unnamed: 0,id,text,creation_date,post_id,user_id,user_display_name,score
641513,641513,51312642,Try this assuming your matrix is stored in `A`...,2015-07-28 19:33:25.633000+00:00,31685078,3250829.0,NaN,1
547146,547146,3383081,I think Frank is asking for instructions on ho...,2010-07-17 05:07:22.650000+00:00,3270170,385115.0,NaN,1
998506,998506,92906537,What is the call to fill_pie() with no data in...,2018-10-26 03:49:53.013000+00:00,53001058,4670239.0,NaN,1


In [9]:
tags_sample.head(3)

,_Id,_TagName,_Count,_ExcerptPostId,_WikiPostId
15857,45720,pretty-urls,157,25923660.0,25923659.0
10027,25258,tournament,87,45217505.0,45217504.0
52298,134973,php-namespaces,3,52681673.0,52681672.0


In [10]:
answers_sample.head(3)

,Unnamed: 0,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,...,parent_id,post_type_id,score,tags,view_count,UnorderedLists,OrderedLists,CodeSnippets,SampleOutput,Images
136969,136969,50676838,NaN,"<p>Actually, for both AAD Graph API and Micros...",NaN,NaN,3,NaN,2018-06-04 08:58:33.090000+00:00,NaN,...,50675653,2,1,NaN,NaN,0.0,0.0,1.0,0.0,0.0
796972,796972,7734649,NaN,<p>You could try:</p>\r\n\r\n<pre><code>text(....,NaN,NaN,0,NaN,2011-10-12 02:33:41.277000+00:00,NaN,...,7734535,2,0,NaN,NaN,0.0,0.0,1.0,0.0,0.0
884854,884854,3875921,NaN,"<p>Found a solution in <a href=""https://stacko...",NaN,NaN,1,NaN,2010-10-06 18:52:54.410000+00:00,NaN,...,2669727,2,0,NaN,NaN,0.0,0.0,1.0,0.0,0.0


In [11]:
questions_sample.head(3)

,Unnamed: 0,Unnamed: 0.1,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,...,seaborn,geospatial,stata,plyr,pie-chart,graphviz,spss,diagram,qlikview,altair
468670,468670,468670,6460450,how does inline functions expose internal data...,"<p>I hear this a lot of times that: ""inline fu...",6460507.0,3,1,NaN,2011-06-23 20:50:49.553000+00:00,...,0,0,0,0,0,0,0,0,0,0
435508,435508,435508,26975575,iOS stack Animations in series,<p>I would like to create a set of animations ...,26975725.0,2,1,NaN,2014-11-17 15:04:44.453000+00:00,...,0,0,0,0,0,0,0,0,0,0
386127,386127,386127,22047085,Using the css3 animation property 'alternate' ...,<p>I've set up 3 divs to animate in a wavelike...,NaN,0,8,NaN,2014-02-26 16:17:31.413000+00:00,...,0,0,0,0,0,0,0,0,0,0


# Tokenizer

In [12]:
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # gets just the txt, without the html tags
    txt = BeautifulSoup(sentence, 'html.parser').get_text()
    
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(txt)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [14]:
# try to run the tokenizer manually to inspect results
# can only work for samples

# helper function, takes in words, and dictionary, and outputs a list representing 
def text2bow(words: List[str], dictionary: Dict[str, int]) -> List[Tuple[int, int]]:
    word_frequences = collections.defaultdict(int)
    for word in words:
        if word not in dictionary:
            dictionary[word] = len(dictionary)
        word_frequences[dictionary[word]] += 1
    return list(word_frequences.items())

def gen_ngrams_questions(questions):
    dictionary = {}
    bow_list = []
    questions_body = questions["body"]
    for q in questions_body:
        q_tokens = spacy_tokenizer(q)
        bow = text2bow(q_tokens, dictionary)
        bow_list.append(bow)
    results = {"bow_representation" : bow_list, "dictionary" : dictionary}
    return(results)

results = gen_ngrams_questions(questions_sample)
print("BOW representations ===================")
print(results["bow_representation"])
print("Dictionary ===================")
print(results["dictionary"])


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Cleaner (Transformer)

In [16]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

# Vectorization Feature Engineering (TF-IDF)

In [17]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
bow_vector

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function spacy_tokenizer at 0x7fb98af184d0>,
                vocabulary=None)

In [30]:
bow_vector_2 = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(5,5))
bow_vector_2

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(5, 5), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function spacy_tokenizer at 0x7fb98af184d0>,
                vocabulary=None)

In [18]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)
tfidf_vector

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function spacy_tokenizer at 0x7fb98af184d0>,
                use_idf=True, vocabulary=None)

# Splitting The Data into Training and Test Sets

In [19]:
from sklearn.model_selection import train_test_split

X = questions_sample["body"] # the features we want to analyze

# TODO replace with real label(s)
questions_sample["tmp"] = questions_sample["accepted_answer_id"] > 0
questions_sample["has_answer"] = questions_sample["tmp"].astype(int)

ylabels = questions_sample["has_answer"] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size = 0.3)

print("train X and y")
print(X_train.shape)
print(y_train.shape)
print("test X and y")
print(X_test.shape)
print(y_test.shape)

train X and y
(4200,)
(4200,)
test X and y
(1800,)
(1800,)


# Creating a Pipeline and Generating the Model

In [38]:
# get the term matrix

# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
vectorizer = tfidf_vector

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer)])

# model generation
wm = pipe.fit_transform(X_train)

tokens = vectorizer.get_feature_names()

In [40]:
print(wm.shape)
print(wm.toarray())

(4200, 84660)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
vectorizer = bow_vector

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', classifier)])

# Eval of Model

In [24]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.5
Logistic Regression Precision: 0.5082304526748971
Logistic Regression Recall: 0.5393013100436681
